# 6.1 매개변수 갱신

최적의 매개변수 찾기 = 최적화 (Optimization)

확률적 경사 하강법 (SGD) : 매개 변수의 기울기를 구해, 기울어진 방향으로 매개 변수 값을 갱신하는 일을 반복해서 최적의 매개 변수를 찾는 과정 

## 6.1.2 확률적 경사 하강법 (SGD)

W = W - r미분(L/W) 

W 는 가중치 매개변수 <br>
r 는 학습률<br>
미분(L/W) 는 W 에 대한 손실함수의 기울기 

기울어진 방향으로 일정거리만 가겠다. 


In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr*grads[key]

* lr : learning rate
* update(params, grads) dictinary 변수로 가중치와 기울기를 저장하고 있음 

## 6.1.3 SGD 의 단점 

anisortropy 비등방성 (방향에 따라 성질, 즉 기울기가 달라지는 함수) 에서는 탐색 경로가 비효율 적이다. 

지그재그로 탐색하는 근본 원인은 기울어진 방향이 본래의 최솟값과 다른 방향을 가리키기 때문이다. 

## 6.1.4 모멘텀
Momentum : 운동량 

v = av - r미분(L/W)
W = W + v

W : 갱신할 가중치 매개변수 <br>
av : 아무런 힘을 받지 않을 때 서서히 하강시킴 (공기저항)<br>
v : 속도 <br>
기울기 방향으로 힘을 받아 물체가 가속됨 


In [2]:
class Momentum:
    def __init__(self, lr = 0.01, momentum =0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]
            

v 는 초기에 아무것도 담지 않음 update 호출 될때 매개변수와 같은 구조로 저장됨 

SGD 에 비하면 지그재그 정도가 덜하다.

## 6.1.5 AdaGrad

학습률 을 정하는 기술 학습률 감소 learning rate decay <br>
학습을 진행하면서 학습률을 점차 줄여가는 방법

AdaGrad 는 매개변수 '전체'의 학습률 값을 일괄적으로 낮추는 것에서 발전시켜 '각각의' 매개변수에 맞춤형 값을 만들어 주는 방식 

개별 매개변수에 적응적(adative) 학습률을 조정하면서 학습을 진행 

h 라는 변수가 등장하여 기존 기울기 값을 제곱하여 계속 더해주고 매개변수가 갱신할 때 1/루트h 를 곱해 학습률을 조정

매개변수의 원소 중에서 많이 움직인 (크게 갱신된) 원소는 학습률이 낮아지게 됨 

과거의 기울기를 제곱해서 계속 더해가서 학습을 진행할수록 갱신 강도가 약해진다. 학습을 계쏙하면 순간 갱신량이 0이되어 전혀 갱신되지 않게 된다. 이를 개선한 RMSProp 는 과거의 모든 기울기를 균일하게 더해가는 것이 아니라, 먼 과거의 기울기는 서서히 잊고 새로운 기울기 정보를 크게 반영하는 지수이동평균 Exponential Moving Average 라 하여, 과거 기울기의 반영규모를 기하급수적으로 감소 시킨다. 

In [2]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key]/(np.sqrt(self.h[key])+1e-7)

마지막에 +1e-7 작은 값을 더하는 부분은 self.h[key] 에 0이 담겨 있다해도 0으로 나누는 사태를 막아준다.

y축 방향은 기울기가 커서 처음에는 크게 움직이지만, 그 큰 움직임에 비례해 갱신 정도도 큰복으로 작아지도록 조정된다. y 축 방향으로 갱신 강도가 약해졌고, 지그재그 움직임이 줄어든다. 